# Задание №3

## Задача

На основе уже знакомого тебе тестового набора данных из первого задания посчитай следующие метрики в тетрадке Jupyter Notebook:
- Delivery rate
- Open rate
- Click to Open rate
- Unsubscribe rate
- Выяви лучшую тему
- И лучший день недели (для планирования рассылок)




## Предобработка данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# прочитаем данные из csv
df = pd.read_csv("data.csv") 
df.head()

,Название рассылки,Название кампании,Направление,Месяц,Дата,Год,Номер недели,День недели,День недели.1,Время,...,Доставлено,Открытия,Клики,Баунсы (Все ошибки),Отписки,UTM Метка,Пользователей на сайте,Воронка продаж. Шаг 1,Воронка продаж. Шаг 2,Воронка продаж. Шаг 3
0,Название рассылки 1,Название кампании 1,Email,Октябрь,27.10.2021,2021,43,3,03-среда,19:24,...,741 750,148 350,17 802,39 039,7 417,Метка 1,16 378,6337,6210,5154
1,Название рассылки 10,Название кампании 10,Email,Ноябрь,05.11.2021,2021,45,5,05-пятница,12:02,...,683 402,123 012,11 071,35 969,6 834,Метка 10,10 296,3558,3096,2539
2,Название рассылки 100,Название кампании 100,Email,Апрель,11.04.2022,2022,15,1,01-понедельник,16:26,...,1 141 344,182 615,15 340,60 071,11 413,Метка 100,13 959,3370,2864,2263
3,Название рассылки 101,Название кампании 101,Email,Апрель,12.04.2022,2022,15,2,02-вторник,16:26,...,1 324 136,264 827,10 328,69 691,13 241,Метка 101,8 986,5116,4277,3208
4,Название рассылки 102,Название кампании 102,Email,Апрель,13.04.2022,2022,15,3,03-среда,16:26,...,1 212 980,218 336,15 720,63 841,12 130,Метка 102,10 847,2817,2287,1670


Приведем к змеиному регистру

In [3]:
df.columns = df.columns.str.lower() 
df.columns = df.columns.str.replace(' ', '_', regex=True).str.replace('.', '_', regex=True)

df = df.rename(columns={'тема_письма_': 'тема_письма'})
df = df.rename(columns={'сегмент_': 'сегмент'})

Проверим типы данных

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   название_рассылки       218 non-null    object
 1   название_кампании       218 non-null    object
 2   направление             218 non-null    object
 3   месяц                   218 non-null    object
 4   дата                    218 non-null    object
 5   год                     218 non-null    int64 
 6   номер_недели            218 non-null    int64 
 7   день_недели             218 non-null    int64 
 8   день_недели_1           218 non-null    object
 9   время                   218 non-null    object
 10  веб-версия              218 non-null    object
 11  тема_письма             218 non-null    object
 12  сегмент                 218 non-null    object
 13  отправлено              218 non-null    object
 14  доставлено              218 non-null    object
 15  открыт

Приведем данные к нужным типам

In [5]:
df['дата'] = pd.to_datetime(df['дата'], format='%d.%m.%Y')

# Преобразование времени
df['время'] = pd.to_timedelta(df['время'] + ':00')

# Преобразование числовых столбцов в int
numeric_columns = ['отправлено', 'доставлено', 'открытия', 'клики', 'баунсы_(все_ошибки)', 'отписки', 'пользователей_на_сайте']
df[numeric_columns] = df[numeric_columns].apply(lambda x: x.str.replace(r'\s+', '', regex=True)).astype(int)
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

# Вывод обновленного датафрейма
df.head()

,название_рассылки,название_кампании,направление,месяц,дата,год,номер_недели,день_недели,день_недели_1,время,...,доставлено,открытия,клики,баунсы_(все_ошибки),отписки,utm_метка,пользователей_на_сайте,воронка_продаж__шаг_1,воронка_продаж__шаг_2,воронка_продаж__шаг_3
0,Название рассылки 1,Название кампании 1,Email,Октябрь,2021-10-27,2021,43,3,03-среда,0 days 19:24:00,...,741750,148350,17802,39039,7417,Метка 1,16378,6337,6210,5154
1,Название рассылки 10,Название кампании 10,Email,Ноябрь,2021-11-05,2021,45,5,05-пятница,0 days 12:02:00,...,683402,123012,11071,35969,6834,Метка 10,10296,3558,3096,2539
2,Название рассылки 100,Название кампании 100,Email,Апрель,2022-04-11,2022,15,1,01-понедельник,0 days 16:26:00,...,1141344,182615,15340,60071,11413,Метка 100,13959,3370,2864,2263
3,Название рассылки 101,Название кампании 101,Email,Апрель,2022-04-12,2022,15,2,02-вторник,0 days 16:26:00,...,1324136,264827,10328,69691,13241,Метка 101,8986,5116,4277,3208
4,Название рассылки 102,Название кампании 102,Email,Апрель,2022-04-13,2022,15,3,03-среда,0 days 16:26:00,...,1212980,218336,15720,63841,12130,Метка 102,10847,2817,2287,1670


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   название_рассылки       218 non-null    object         
 1   название_кампании       218 non-null    object         
 2   направление             218 non-null    object         
 3   месяц                   218 non-null    object         
 4   дата                    218 non-null    datetime64[ns] 
 5   год                     218 non-null    int64          
 6   номер_недели            218 non-null    int64          
 7   день_недели             218 non-null    int64          
 8   день_недели_1           218 non-null    object         
 9   время                   218 non-null    timedelta64[ns]
 10  веб-версия              218 non-null    object         
 11  тема_письма             218 non-null    object         
 12  сегмент                 218 non-null

**Вывод:** данные готовы к анализу и расчету метрик.

## Расчет метрик

In [7]:
# Округление значений и вычисление процентов
df['delivery_rate_%'] = round(df['доставлено'] / df['отправлено'] * 100,2)
df['open_rate_%'] = round(df['открытия'] / df['доставлено'] * 100, 2)
df['сlick_to_open_rate_%'] = round(df['клики'] / df['открытия'] * 100, 2)
df['unsubscribe_rate_%'] = round(df['отписки'] / df['доставлено'] * 100, 2)

# Вывод первых 5 строк
df.head(5)

,название_рассылки,название_кампании,направление,месяц,дата,год,номер_недели,день_недели,день_недели_1,время,...,отписки,utm_метка,пользователей_на_сайте,воронка_продаж__шаг_1,воронка_продаж__шаг_2,воронка_продаж__шаг_3,delivery_rate_%,open_rate_%,сlick_to_open_rate_%,unsubscribe_rate_%
0,Название рассылки 1,Название кампании 1,Email,Октябрь,2021-10-27,2021,43,3,03-среда,0 days 19:24:00,...,7417,Метка 1,16378,6337,6210,5154,95.0,20.0,12.0,1.0
1,Название рассылки 10,Название кампании 10,Email,Ноябрь,2021-11-05,2021,45,5,05-пятница,0 days 12:02:00,...,6834,Метка 10,10296,3558,3096,2539,95.0,18.0,9.0,1.0
2,Название рассылки 100,Название кампании 100,Email,Апрель,2022-04-11,2022,15,1,01-понедельник,0 days 16:26:00,...,11413,Метка 100,13959,3370,2864,2263,95.0,16.0,8.4,1.0
3,Название рассылки 101,Название кампании 101,Email,Апрель,2022-04-12,2022,15,2,02-вторник,0 days 16:26:00,...,13241,Метка 101,8986,5116,4277,3208,95.0,20.0,3.9,1.0
4,Название рассылки 102,Название кампании 102,Email,Апрель,2022-04-13,2022,15,3,03-среда,0 days 16:26:00,...,12130,Метка 102,10847,2817,2287,1670,95.0,18.0,7.2,1.0


In [8]:
means = round(df[['delivery_rate_%', 'open_rate_%', 'сlick_to_open_rate_%', 'unsubscribe_rate_%']].mean(),2)

# Красивый вывод средних значений
print("Средние значения:")
for column, mean_value in means.items():
    print(f"{column}: {mean_value}%")

Средние значения:
delivery_rate_%: 97.65%
open_rate_%: 13.76%
сlick_to_open_rate_%: 8.12%
unsubscribe_rate_%: 2.47%


**Вывод**: 

- Метрики рассчитаны. 
- Средние значения:

  - delivery_rate_%: 97.65%
  - open_rate_%: 13.76%
  - сlick_to_open_rate_%: 8.12%
  - unsubscribe_rate_%: 2.47%

## Лучшая тема

Чтобы найти лучшую тему необходимо отсортировать его в порядке убывания метрик. Сравним темы и найдем максимальные по open rate, click to open rate, delivery rate и минимальный unsubscribe rate.

In [9]:
grouped = df.groupby('название_рассылки').mean(numeric_only=True)

# Сортировка по указанным столбцам
sorted_df = grouped.sort_values(by=['delivery_rate_%', 'open_rate_%', 'unsubscribe_rate_%', 'сlick_to_open_rate_%'], ascending=[False, False, True, False])

# Вывод результатов
sorted_df.head(5)

,год,номер_недели,день_недели,отправлено,доставлено,открытия,клики,баунсы_(все_ошибки),отписки,пользователей_на_сайте,воронка_продаж__шаг_1,воронка_продаж__шаг_2,воронка_продаж__шаг_3,delivery_rate_%,open_rate_%,сlick_to_open_rate_%,unsubscribe_rate_%
название_рассылки,,,,,,,,,,,,,,,,,
Название рассылки 204,2022.0,2.0,1.0,542459.0,534322.0,96178.0,8079.0,8137.0,1336.0,7837.0,3269.0,2778.0,2195.0,98.5,18.0,8.4,0.25
Название рассылки 189,2021.0,49.0,4.0,2131798.0,2099821.0,377968.0,27214.0,31977.0,12179.0,25309.0,8747.0,7610.0,6012.0,98.5,18.0,7.2,0.58
Название рассылки 57,2022.0,3.0,1.0,1722650.0,1696810.0,305426.0,27488.0,25840.0,15271.0,23915.0,13617.0,11575.0,9144.0,98.5,18.0,9.0,0.90
Название рассылки 9,2021.0,45.0,5.0,635111.0,625584.0,112605.0,9459.0,9527.0,5630.0,6527.0,1695.0,1376.0,1087.0,98.5,18.0,8.4,0.90
Название рассылки 73,2022.0,8.0,5.0,1026345.0,1010950.0,181971.0,13102.0,15395.0,9099.0,12054.0,4664.0,3899.0,3080.0,98.5,18.0,7.2,0.90


**Вывод:** наилучшие показатели у рассылки 204

## Лучший день для планирования рассылок

In [10]:
# Группировка по дню недели и вычисление средних значений для каждого дня
grouped = df.groupby('день_недели').mean(numeric_only=True)

# Определение лучшего дня по наивысшим показателям
best_day = grouped.idxmax(numeric_only=True)

# Вывод результата
print("Лучший день для планирования рассылок: ", best_day[0])

Лучший день для планирования рассылок:  4


**Вывод:** лучший день для планирования рассылок:  четверг

## Выводы

- Средние значения метрик:

   - delivery_rate_%: 97.65%
   - open_rate_%: 13.76%
   - сlick_to_open_rate_%: 8.12%
   - unsubscribe_rate_%: 2.47%
   
- Наилучшие показатели у рассылки 204
- Лучший день для планирования рассылок: четверг